In [18]:
import numpy as np
import math
import  re
import time
from tqdm import tqdm_notebook 
from collections import defaultdict
from sklearn.cross_validation import train_test_split
from functools import reduce

Чтение полученного разбиения на кластеры и добавление в словарь

In [3]:
f = open('fout_exemplars.txt','r')

#соответсвие между юзерами и кластремами, ключ: user_id
id_cluster = defaultdict()

for line in tqdm_notebook(f):
    splited_line = re.split(r'\t',line)
    _id = int(splited_line[0])
    n_cluster = int(splited_line[1])
    id_cluster[_id] = n_cluster


A Jupyter Widget

In [19]:
len(set(id_cluster.values())) ## число полученых кластеров

41820

Чтение локаций и создание словаря, хранящего чекины каждого юзера

In [20]:
f = open("Gowalla_totalCheckins.txt", "r")

user_loc = defaultdict(list)

for line in tqdm_notebook(f):
    splited_line = re.split(r'\t',line)
    user_id = int(splited_line[0])
    location_id = int(splited_line[4])
    #добавление в словарь
    user_loc[user_id].append(location_id)

    
    

A Jupyter Widget

Разделение юзеров на тренировочных и тестовых(кому будут строиться рекомендации)

In [17]:

user_ids = user_loc.keys() #только те юзеры, которые имеют локации
train_ids,test_ids = train_test_split(list(user_ids), test_size = 0.10)


In [21]:
len(user_ids) # юзеры, имеющие локации

107092

Создание словаря, хранящего чекины всех юзеров (из train), входящих в данный кластер.

key = номер кластера

In [8]:
cluster_loc = defaultdict(list)

for i in train_ids:
    cluster = id_cluster[i]
    locations = user_loc[i]
    for location in locations:
        cluster_loc[cluster].append(location)

In [22]:
len(cluster_loc) #число кластеров,попавших в train

34426

Подсчет качества рекомендаций

In [10]:

_sum = 0

for _id in tqdm_notebook(test_ids):
    start = time.time()
   
    cluster = id_cluster[_id]
    #полный список предсказанных локаций (несортированных по частоте встречаемости)
    predict_loc = cluster_loc[cluster]

    
    #сортировка по частоте встречаемости
    predict_loc = sorted(predict_loc,key=predict_loc.count,reverse=True)
    
    #оставить единственный экземпляр, сохраняя порядок сортировки
    predict_loc = reduce(lambda l,x:l+[x] if x not in l else l, predict_loc,[])
   
    
    #дополнить "ошибками" если меньше 10 локаций в кластере
    while len(predict_loc) < 10:
        predict_loc.append(-1)   
    
    #if len(predict_loc)>20000:
     #   print("time: ", time.time() - start)
        
    # изменить значение в словаре, оставив только 10 популярных локаций     
    cluster_loc[cluster] = predict_loc[:10] 
     
    #истинные локации юзера
    current_loc = user_loc[_id]
    
    #прибавление числа пересечений 10 предсказанных с реальными локациями юзера
    _sum+= len(set(predict_loc[:10])&set(current_loc))
    #print("sum = ", _sum)
    

A Jupyter Widget

In [23]:
#максимальное число пересечений для всех тестовых юзеров
test_sum = 10 * len(test_ids) 

#отношение полученной суммы к максимально возможной
_sum / test_sum


0.05122315592903828